In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
import warnings
import os
import sys
from tqdm import tqdm
import cv2
import math

ModuleNotFoundError: No module named 'torchsummary'

# Import Stuff & Setup

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

NameError: name 'os' is not defined

In [ ]:
RESOLUTION = 64
block_frame = 8
frame_interval = 15

In [ ]:
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

In [ ]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    # transforms.Lambda(lambda x: x / 255.),
    transforms.Resize((RESOLUTION, RESOLUTION))
])


# Utils

# Dataset

In [ ]:
# DATA_SOURCE = {
#     "Abuse": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Abuse",
#     "Arrest": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arrest",
#     "Arson": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arson",
#     "Assault": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Assault",
#     "Burglary": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Burglary",
#     "Explosion": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Explosion",
#     "Fighting": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Fighting",
#     "Normal" : "E:\\kaggle\\input\\crimeufcdataset\\Anomaly_Dataset\\Anomaly_Videos\\Normal-Videos-Part-1"
# }

DATA_SOURCE = {
    "Abuse": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Abuse",
    "Arrest": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Arrest",
    "Arson": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Arson",
    "Assault": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Assault",
    "Burglary": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Burglary",
    "Explosion" : "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Explosion",
    "Fighting" : "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Fighting",
    "RoadAccidents" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\RoadAccidents",
    "Robbery" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\Robbery",
    "Shooting" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\Shooting",
    "Shoplifting" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Shoplifting",
    "Stealing" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Stealing",
    "Vandalism" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Vandalism",
    "Normal" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Training-Normal-Videos-Part-1",
}

TEST_SOURCE = {
    "Normal" : r"E:\kaggle\input\Anomaly-Detection-Dataset\Testing_Normal_Videos_Anomaly"
}



# DATA_SOURCE = {
#     "Abuse": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Abusetest",
#     "Arrest": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arresttest",
# }



# for test

In [ ]:
# DATA_SOURCE = {
#     "Abuse": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Abuse",
#     "Arrest": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arrest",
#     "Arson": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arson",
#     "Assault": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Assault",
#     "Burglary": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Burglary",
#     "Explosion": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Explosion",
#     "Fighting": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Fighting",
#     "Normal" : "E:\\kaggle\\input\\crimeufcdataset\\Anomaly_Dataset\\Anomaly_Videos\\Normal-Videos-Part-1"
# }

DATA_SOURCE = {
    "Abuse": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Abuse",
    "Arrest": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Arrest",
    "Arson": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Arson",
    "Assault": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-1\\Assault",
    "Burglary": "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Burglary",
    "Explosion" : "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Explosion",
    "Fighting" : "E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-2\\Fighting",
    "RoadAccidents" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\RoadAccidents",
    "Robbery" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\Robbery",
    "Shooting" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-3\\Shooting",
    "Shoplifting" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Shoplifting",
    "Stealing" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Stealing",
    "Vandalism" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Anomaly-Videos-Part-4\\Vandalism",
    "Normal" :"E:\\kaggle\\input\\Anomaly-Detection-Dataset\\Training-Normal-Videos-Part-1",
}

# TEST_SOURCE = {
#     "Normal" : r"E:\kaggle\input\Anomaly-Detection-Dataset\Testing_Normal_Videos_Anomaly"
# }


In [ ]:
import numpy as np
txt_train = r"E:\kaggle\input\Anomaly-Detection-Dataset\Anomaly_Train.txt"
train_dic = {}
with open(txt_train, 'r') as file:
    for line in file:
        parts = line.strip().split('/')
        train_dic[parts[1]] = parts[0]
    
txt_test = r"E:\kaggle\input\Anomaly-Detection-Dataset\Temporal_Anomaly_Annotation_for_Testing_Videos.txt"
test_dic = {}
with open(txt_test, 'r') as file:
    for line in file:
        parts = line.strip().split()
        video_name = parts[0]
        event_name = parts[1]
        start_frame1 = int(parts[2])
        end_frame1 = int(parts[3])
        start_frame2 = int(parts[4])
        end_frame2 = int(parts[5])
        test_dic[video_name] = [event_name, start_frame1, end_frame1, start_frame2, end_frame2]


In [ ]:
listtest = txt_test.split('\\')
print(listtest)

In [ ]:
import os
# 경로가 존재하는지 확인합니다.
if os.path.exists(txt_test):
    print("경로가 존재합니다.")
    # 디렉터리 내의 파일 목록을 출력합니다.
else:
    print("지정된 경로를 찾을 수 없습니다.")


In [ ]:
import os
for directory_path in TEST_SOURCE.items():
# 경로가 존재하는지 확인합니다.
    if os.path.exists(directory_path[1]):
        print("경로가 존재합니다.")
        # 디렉터리 내의 파일 목록을 출력합니다.
        print("디렉터리 내용:", os.listdir(directory_path[1]))
    else:
        print("지정된 경로를 찾을 수 없습니다.")


In [ ]:
event_name, start_frame1, end_frame1, start_frame2, end_frame2 = test_dic[""]
print(frame_count, start_frame1, start_frame2, end_frame1, end_frame2)

In [ ]:
class CrimeDataset(Dataset):
    train = True
    def __init__(self, __train=True):
        train = __train
        self._data = []
        self._labels = []
        self._frame_interval = frame_interval
        
        print(f"Loading {'train' if train else 'test' } dataset...")

        if train:
            for label, data_path in DATA_SOURCE.items():
                print(f"Loading Label {label}...")
            
                for file in tqdm(os.listdir(data_path)):
                    if file.endswith(".mp4") and file in train_dic:
                        path = os.path.join(data_path, file)
                        data, labels = self._parse_file(path, label)
                    
                        self._data.extend(data)
                        self._labels.extend(labels)
        else:
            for label, data_path in DATA_SOURCE.items():
                print(f"Loading Label {label}...")
            
                for file in tqdm(os.listdir(data_path)):
                    if file.endswith(".mp4") and file in test_dic:
                        path = os.path.join(data_path, file)
                        data, labels = self._parse_file(path, label)
                
                    
                        self._data.extend(data)
                        self._labels.extend(labels)
                    
        print(f"Finished loading {'train' if train else 'test' } dataset... Loaded  {len(self._data)} images.")
    
    def _parse_file(self, __path, __label):
        if not os.path.exists(__path):
            return [], []
        
        data = []
        labels = []
        
        cap = cv2.VideoCapture(__path)
        
        frames = []
        frame_count = 0
        success, image = cap.read()
        while success:
            try:
                if True:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
                    Y, U, V = cv2.split(image)
                    image = transformer(Y)
                    frames.append(image)
                    
                    if len(frames) == block_frame:
                        if train:
                            stacked_frames = torch.stack(frames)
                            stacked_frames = stacked_frames.permute(1, 0, 2, 3)
                            data.append(stacked_frames)
                            labels.append(__label)
                            frames = frames[3:]
                        else:
                            stacked_frames = torch.stack(frames)
                            stacked_frames = stacked_frames.permute(1, 0, 2, 3)
                            data.append(stacked_frames)
                            filepath_list = __path.split('\\')
                            event_name, start_frame1, end_frame1, start_frame2, end_frame2 = test_dic[filepath_list[-1]]
                            if (frame_count >= start_frame1 and frame_count <= end_frame1) or (frame_count >= start_frame2 and frame_count <= end_frame2):
                                labels.append(__label)
                            else:
                                labels.append("Normal")
                            frames = frames[3:]
            except Exception as e:
                print(f"Error loading file {__path}: {e}")
            
            count = 0
            while success and count < self._frame_interval:
                success, image = cap.read()
                frame_count +=1
                count += 1
                
        if len(frames) > 0 and len(frames) < block_frame:
            while len(frames) < block_frame:
                frames.append(frames[-1])
            stacked_frames = torch.stack(frames)
            stacked_frames = stacked_frames.permute(1, 0, 2, 3)
            data.append(stacked_frames)
            if train:
                labels.append(__label)
            else:
                if (frame_count >= start_frame1 and frame_count <= end_frame1) or (frame_count >= start_frame2 and frame_count <= end_frame2):
                    labels.append(__label)
                else:
                    labels.append("Normal")       
        return data, labels
        
    
    def label_str2id(self, __label):
        labels = DATA_SOURCE.keys()
        return list(labels).index(__label)

    def label_id2str(self, __label):
        labels = DATA_SOURCE.keys()
        return list(labels)[__label]
        
    def __len__(self):
        return len(self._labels)

    def __getitem__(self, __idx):
        data = self._data[__idx]
        label = self._labels[__idx]
        return data, torch.tensor([self.label_str2id(label)])

In [ ]:
#train = True
#train_dataset = CrimeDataset(True)
train = False
test_dataset = CrimeDataset(False)

In [ ]:
print(test_dataset._labels)

In [ ]:
print(train_dataset._data[0].shape)
plot_randomly_form_dataset(train_dataset)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
print(len(train_dataset))
# print(len(train_dataset._data))
# print(len(train_dataset._labels))
print(train_dataset._data[0].shape)
print(train_dataset[0][1])
print(train_dataset[3][0].shape)
plot_sequence_form_dataset(train_dataset, 4)

# Model

In [30]:
convnum = 0
convadd = 0
convin = []
convint = []
convoup = []
convkernel = []
convstrid = []
convpadding = []
convgroup = []

In [81]:
from typing import Any, Callable, Dict, List, Mapping, Optional, Tuple, Union

import torch
import torch.nn as nn
import torch.nn.functional as F


def make_divisible(
        value: float,
        divisor: int,
        min_value: Optional[float] = None,
        round_down_protect: bool = True,
    ) -> int:
    """
    This function is copied from here 
    "https://github.com/tensorflow/models/blob/master/official/vision/modeling/layers/nn_layers.py"
    
    This is to ensure that all layers have channels that are divisible by 8.

    Args:
        value: A `float` of original value.
        divisor: An `int` of the divisor that need to be checked upon.
        min_value: A `float` of  minimum value threshold.
        round_down_protect: A `bool` indicating whether round down more than 10%
        will be allowed.

    Returns:
        The adjusted value in `int` that is divisible against divisor.
    """
    if min_value is None:
        min_value = divisor
    new_value = max(min_value, int(value + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if round_down_protect and new_value < 0.9 * value:
        new_value += divisor
    return int(new_value)

def conv_2d(inp, oup, kernel_size=3, stride=1, groups=1, bias=False, norm=True, act=True):
    conv = nn.Sequential()
    padding = (kernel_size - 1) // 2
    conv.add_module('conv', nn.Conv3d(inp, oup, kernel_size, stride, padding, bias=bias, groups=groups))
    if norm:
        conv.add_module('BatchNorm2d', nn.BatchNorm3d(oup))
    if act:
        conv.add_module('Activation', nn.ReLU6())
    convint.append(inp)
    convoup.append(oup)
    convkernel.append(kernel_size)
    convstrid.append(stride)
    convpadding.append(padding)
    convgroup.append(groups)
    #convnum += 1
    
    return conv

class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, act=False, squeeze_excitation=False):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]
        hidden_dim = int(round(inp * expand_ratio))
        self.block = nn.Sequential()
        if expand_ratio != 1:
            self.block.add_module('exp_1x1', conv_2d(inp, hidden_dim, kernel_size=3, stride=stride))
        if squeeze_excitation:
            self.block.add_module('conv_3x3', conv_2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, groups=hidden_dim))
        self.block.add_module('red_1x1', conv_2d(hidden_dim, oup, kernel_size=1, stride=1, act=act))
        self.use_res_connect = self.stride == 1 and inp == oup

    def forward(self, x):
        if self.use_res_connect:
            return x + self.block(x)
        else:
            return self.block(x)

class UniversalInvertedBottleneckBlock(nn.Module):
    def __init__(self, 
            inp, 
            oup, 
            start_dw_kernel_size, 
            middle_dw_kernel_size, 
            middle_dw_downsample,
            stride,
            expand_ratio
        ):
        """An inverted bottleneck block with optional depthwises.
        Referenced from here https://github.com/tensorflow/models/blob/master/official/vision/modeling/layers/nn_blocks.py
        """
        super().__init__()
        # Starting depthwise conv.
        self.start_dw_kernel_size = start_dw_kernel_size
        if self.start_dw_kernel_size:            
            stride_ = stride if not middle_dw_downsample else 1
            self._start_dw_ = conv_2d(inp, inp, kernel_size=start_dw_kernel_size, stride=stride_, groups=inp, act=False)
        # Expansion with 1x1 convs.
        expand_filters = make_divisible(inp * expand_ratio, 8)
        self._expand_conv = conv_2d(inp, expand_filters, kernel_size=1)
        # Middle depthwise conv.
        self.middle_dw_kernel_size = middle_dw_kernel_size
        if self.middle_dw_kernel_size:
            stride_ = stride if middle_dw_downsample else 1
            self._middle_dw = conv_2d(expand_filters, expand_filters, kernel_size=middle_dw_kernel_size, stride=stride_, groups=expand_filters)
        # Projection with 1x1 convs.
        self._proj_conv = conv_2d(expand_filters, oup, kernel_size=1, stride=1, act=False)
        
        # Ending depthwise conv.
        # this not used
        # _end_dw_kernel_size = 0
        # self._end_dw = conv_2d(oup, oup, kernel_size=_end_dw_kernel_size, stride=stride, groups=inp, act=False)
        
    def forward(self, x):
        if self.start_dw_kernel_size:
            x = self._start_dw_(x)
            # print("_start_dw_", x.shape)
        x = self._expand_conv(x)
        # print("_expand_conv", x.shape)
        if self.middle_dw_kernel_size:
            x = self._middle_dw(x)
            # print("_middle_dw", x.shape)
        x = self._proj_conv(x)
        # print("_proj_conv", x.shape)
        return x

class MultiQueryAttentionLayerWithDownSampling(nn.Module):
    def __init__(self, inp, num_heads, key_dim, value_dim, query_h_strides, query_w_strides, kv_strides, dw_kernel_size=3, dropout=0.0):
        """Multi Query Attention with spatial downsampling.
        Referenced from here https://github.com/tensorflow/models/blob/master/official/vision/modeling/layers/nn_blocks.py

        3 parameters are introduced for the spatial downsampling:
        1. kv_strides: downsampling factor on Key and Values only.
        2. query_h_strides: vertical strides on Query only.
        3. query_w_strides: horizontal strides on Query only.

        This is an optimized version.
        1. Projections in Attention is explict written out as 1x1 Conv2D.
        2. Additional reshapes are introduced to bring a up to 3x speed up.
        """
        super().__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.value_dim = value_dim
        self.query_h_strides = query_h_strides
        self.query_w_strides = query_w_strides
        self.kv_strides = kv_strides
        self.dw_kernel_size = dw_kernel_size
        self.dropout = dropout

        self.head_dim = key_dim // num_heads

        if self.query_h_strides > 1 or self.query_w_strides > 1:
            self._query_downsampling_norm = nn.BatchNorm2d(inp)
        self._query_proj = conv_2d(inp, num_heads*key_dim, 1, 1, norm=False, act=False)
        
        if self.kv_strides > 1:
            self._key_dw_conv = conv_2d(inp, inp, dw_kernel_size, kv_strides, groups=inp, norm=True, act=False)
            self._value_dw_conv = conv_2d(inp, inp, dw_kernel_size, kv_strides, groups=inp, norm=True, act=False)
        self._key_proj = conv_2d(inp, key_dim, 1, 1, norm=False, act=False)
        self._value_proj = conv_2d(inp, key_dim, 1, 1, norm=False, act=False)

        self._output_proj = conv_2d(num_heads*key_dim, inp, 1, 1, norm=False, act=False)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        batch_size, seq_length, _, _ = x.size()
        if self.query_h_strides > 1 or self.query_w_strides > 1:
            q = F.avg_pool2d(self.query_h_stride, self.query_w_stride)
            q = self._query_downsampling_norm(q)
            q = self._query_proj(q)
        else:
            q = self._query_proj(x)
        px = q.size(2)
        q = q.view(batch_size, self.num_heads, -1, self.key_dim) # [batch_size, num_heads, seq_length, key_dim]

        if self.kv_strides > 1:
            k = self._key_dw_conv(x)
            k = self._key_proj(k)
            v = self._value_dw_conv(x)
            v = self._value_proj(v)          
        else:
            k = self._key_proj(x)
            v = self._value_proj(x)
        k = k.view(batch_size, 1, self.key_dim, -1) # [batch_size, 1, key_dim, seq_length]
        v = v.view(batch_size, 1, -1, self.key_dim) # [batch_size, 1, seq_length, key_dim]

        # calculate attn score
        attn_score = torch.matmul(q, k) / (self.head_dim ** 0.5)
        attn_score = self.dropout(attn_score)
        attn_score = F.softmax(attn_score, dim=-1)

        context = torch.matmul(attn_score, v)
        context = context.view(batch_size, self.num_heads * self.key_dim, px, px)
        output = self._output_proj(context)
        return output

class MNV4LayerScale(nn.Module):
    def __init__(self, inp, init_value):
        """LayerScale as introduced in CaiT: https://arxiv.org/abs/2103.17239
        Referenced from here https://github.com/tensorflow/models/blob/master/official/vision/modeling/layers/nn_blocks.py
        
        As used in MobileNetV4.

        Attributes:
            init_value (float): value to initialize the diagonal matrix of LayerScale.
        """
        super().__init__()
        self.init_value = init_value
        self._gamma = nn.Parameter(self.init_value * torch.ones(inp, 1, 1))
    
    def forward(self, x):
        return x * self._gamma

class MultiHeadSelfAttentionBlock(nn.Module):
    def __init__(
            self, 
            inp,
            num_heads, 
            key_dim,  
            value_dim, 
            query_h_strides, 
            query_w_strides, 
            kv_strides,
            use_layer_scale,
            use_multi_query, 
            use_residual = True
        ):
        super().__init__()
        self.query_h_strides = query_h_strides
        self.query_w_strides = query_w_strides
        self.kv_strides = kv_strides
        self.use_layer_scale = use_layer_scale
        self.use_multi_query = use_multi_query
        self.use_residual = use_residual

        self._input_norm = nn.BatchNorm2d(inp)
        if self.use_multi_query:
            self.multi_query_attention = MultiQueryAttentionLayerWithDownSampling(
                inp, num_heads, key_dim, value_dim, query_h_strides, query_w_strides, kv_strides
            )
        else:
            self.multi_head_attention = nn.MultiheadAttention(inp, num_heads, kdim=key_dim)
        
        if self.use_layer_scale:
            self.layer_scale_init_value = 1e-5
            self.layer_scale = MNV4LayerScale(inp, self.layer_scale_init_value) 
    
    def forward(self, x):
        # Not using CPE, skipped
        # input norm
        shortcut = x
        x = self._input_norm(x)
        # multi query
        if self.use_multi_query:
            x = self.multi_query_attention(x)
        else:
            x = self.multi_head_attention(x, x)
        # layer scale
        if self.use_layer_scale:
            x = self.layer_scale(x)
        # use residual
        if self.use_residual:
            x = x + shortcut
        return x

def build_blocks(layer_spec):
    if not layer_spec.get('block_name'):
        return nn.Sequential()
    block_names = layer_spec['block_name']
    layers = nn.Sequential()
    if block_names == "convbn":
        schema_ = ['inp', 'oup', 'kernel_size', 'stride']
        for i in range(layer_spec['num_blocks']):
            args = dict(zip(schema_, layer_spec['block_specs'][i]))
            layers.add_module(f"convbn_{i}", conv_2d(**args))
    elif block_names == "uib":
        schema_ =  ['inp', 'oup', 'start_dw_kernel_size', 'middle_dw_kernel_size', 'middle_dw_downsample', 'stride', 'expand_ratio', 'mhsa']
        for i in range(layer_spec['num_blocks']):
            args = dict(zip(schema_, layer_spec['block_specs'][i]))
            mhsa = args.pop("mhsa") if "mhsa" in args else 0
            layers.add_module(f"uib_{i}", UniversalInvertedBottleneckBlock(**args))
            if mhsa:
                mhsa_schema_ = [
                    "inp", "num_heads", "key_dim", "value_dim", "query_h_strides", "query_w_strides", "kv_strides", 
                    "use_layer_scale", "use_multi_query", "use_residual"
                ]
                args = dict(zip(mhsa_schema_, [args['oup']] + (mhsa)))
                layers.add_module(f"mhsa_{i}", MultiHeadSelfAttentionBlock(**args))
    elif block_names == "fused_ib":
        schema_ = ['inp', 'oup', 'stride', 'expand_ratio', 'act']
        for i in range(layer_spec['num_blocks']):
            args = dict(zip(schema_, layer_spec['block_specs'][i]))
            layers.add_module(f"fused_ib_{i}", InvertedResidual(**args))
    else:
        raise NotImplementedError
    return layers


class MobileNetV4(nn.Module):
    def __init__(self, model):
        # MobileNetV4ConvSmall  MobileNetV4ConvMedium  MobileNetV4ConvLarge
        # MobileNetV4HybridMedium  MobileNetV4HybridLarge
        """Params to initiate MobilenNetV4
        Args:
            model : support 5 types of models as indicated in 
            "https://github.com/tensorflow/models/blob/master/official/vision/modeling/backbones/mobilenet.py"        
        """
        super().__init__()
        assert model in MODEL_SPECS.keys()
        self.model = model
        self.spec = MODEL_SPECS[self.model]
       
        # conv0
        self.conv0 = build_blocks(self.spec['conv0'])
        # layer1
        self.layer1 = build_blocks(self.spec['layer1'])
        # layer2
        self.layer2 = build_blocks(self.spec['layer2'])
        # layer3
        self.layer3 = build_blocks(self.spec['layer3'])
        # layer4
        self.layer4 = build_blocks(self.spec['layer4'])
        # layer5   
        self.layer5 = build_blocks(self.spec['layer5'])       
               
    def forward(self, x):
        x0 = self.conv0(x)
        print("conv2d", x0.shape)
        x1 = self.layer1(x0)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        x5 = self.layer5(x4)
        x5 = nn.functional.adaptive_avg_pool2d(x5, 1 )
        return [x1, x2, x3, x4, x5]
       


In [82]:

MNV4ConvSmall_BLOCK_SPECS = {
    "conv0": {
        "block_name": "convbn",
        "num_blocks": 1,
        "block_specs": [
            [1, 32, 3, 2]
        ]
    },
    "layer1": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [32, 32, 3, 2],
            [32, 32, 1, 1]
        ]
    },
    "layer2": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [32, 96, 3, 2],
            [96, 64, 1, 1]
        ]
    },
    "layer3": {
        "block_name": "uib",
        "num_blocks": 6,
        "block_specs": [
            [64, 96, 5, 5, True, 2, 3],
            [96, 96, 0, 3, True, 1, 2],
            [96, 96, 0, 3, True, 1, 2],
            [96, 96, 0, 3, True, 1, 2],
            [96, 96, 0, 3, True, 1, 2],
            [96, 96, 3, 0, True, 1, 4],
        ]
    },
    "layer4": {
        "block_name": "uib",
        "num_blocks": 6,
        "block_specs": [
            [96,  128, 3, 3, True, 2, 6],
            [128, 128, 5, 5, True, 1, 4],
            [128, 128, 0, 5, True, 1, 4],
            [128, 128, 0, 5, True, 1, 3],
            [128, 128, 0, 3, True, 1, 4],
            [128, 128, 0, 3, True, 1, 4],
        ]
    },  
    "layer5": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [128, 960, 1, 1],
            [960, 1280, 1, 1]
        ]
    }
}

MNV4ConvMedium_BLOCK_SPECS = {
    "conv0": {
        "block_name": "convbn",
        "num_blocks": 1,
        "block_specs": [
            [1, 32, 3, 2]
        ]
    },
    "layer1": {
        "block_name": "fused_ib",
        "num_blocks": 1,
        "block_specs": [
            [32, 48, 2, 4.0, True]
        ]
    },
    "layer2": {
        "block_name": "uib",
        "num_blocks": 2,
        "block_specs": [
            [48, 80, 3, 5, True, 2, 4],
            [80, 80, 3, 3, True, 1, 2]
        ]
    },
    "layer3": {
        "block_name": "uib",
        "num_blocks": 8,
        "block_specs": [
            [80,  160, 3, 5, True, 2, 6],
            [160, 160, 3, 3, True, 1, 4],
            [160, 160, 3, 3, True, 1, 4],
            [160, 160, 3, 5, True, 1, 4],
            [160, 160, 3, 3, True, 1, 4],
            [160, 160, 3, 0, True, 1, 4],
            [160, 160, 0, 0, True, 1, 2],
            [160, 160, 3, 0, True, 1, 4]
        ]
    },
    "layer4": {
        "block_name": "uib",
        "num_blocks": 11,
        "block_specs": [
            [160, 256, 5, 5, True, 2, 6],
            [256, 256, 5, 5, True, 1, 4],
            [256, 256, 3, 5, True, 1, 4],
            [256, 256, 3, 5, True, 1, 4],
            [256, 256, 0, 0, True, 1, 4],
            [256, 256, 3, 0, True, 1, 4],
            [256, 256, 3, 5, True, 1, 2],
            [256, 256, 5, 5, True, 1, 4],
            [256, 256, 0, 0, True, 1, 4],
            [256, 256, 0, 0, True, 1, 4],
            [256, 256, 5, 0, True, 1, 2]
        ]
    },  
    "layer5": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [256, 960, 1, 1],
            [960, 1280, 1, 1]
        ]
    }
}

MNV4ConvLarge_BLOCK_SPECS = {
    "conv0": {
        "block_name": "convbn",
        "num_blocks": 1,
        "block_specs": [
            [1, 24, 3, 2]
        ]
    },
    "layer1": {
        "block_name": "fused_ib",
        "num_blocks": 1,
        "block_specs": [
            [24, 48, 2, 4.0, True]
        ]
    },
    "layer2": {
        "block_name": "uib",
        "num_blocks": 2,
        "block_specs": [
            [48, 96, 3, 5, True, 2, 4],
            [96, 96, 3, 3, True, 1, 4]
        ]
    },
    "layer3": {
        "block_name": "uib",
        "num_blocks": 11,
        "block_specs": [
            [96,  192, 3, 5, True, 2, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 5, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 3, 0, True, 1, 4]
        ]
    },
    "layer4": {
        "block_name": "uib",
        "num_blocks": 13,
        "block_specs": [
            [192, 512, 5, 5, True, 2, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 3, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 3, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4]
        ]
    },  
    "layer5": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [512, 960, 1, 1],
            [960, 1280, 1, 1]
        ]
    }
}

def mhsa(num_heads, key_dim, value_dim, px):
    if px == 24:
        kv_strides = 2
    elif px == 12:
        kv_strides = 1
    query_h_strides = 1 
    query_w_strides = 1 
    use_layer_scale = True 
    use_multi_query = True
    use_residual = True
    return [
        num_heads, key_dim, value_dim, query_h_strides, query_w_strides, kv_strides, 
        use_layer_scale, use_multi_query, use_residual
    ]

MNV4HybridConvMedium_BLOCK_SPECS = {
    "conv0": {
        "block_name": "convbn",
        "num_blocks": 1,
        "block_specs": [
            [3, 32, 3, 2]
        ]
    },
    "layer1": {
        "block_name": "fused_ib",
        "num_blocks": 1,
        "block_specs": [
            [32, 48, 2, 4.0, True]
        ]
    },
    "layer2": {
        "block_name": "uib",
        "num_blocks": 2,
        "block_specs": [
            [48, 80, 3, 5, True, 2, 4],
            [80, 80, 3, 3, True, 1, 2]
        ]
    },
    "layer3": {
        "block_name": "uib",
        "num_blocks": 8,
        "block_specs": [
            [80,  160, 3, 5, True, 2, 6],
            [160, 160, 0, 0, True, 1, 2],
            [160, 160, 3, 3, True, 1, 4],
            [160, 160, 3, 5, True, 1, 4, mhsa(4, 64, 64, 24)],
            [160, 160, 3, 3, True, 1, 4, mhsa(4, 64, 64, 24)],
            [160, 160, 3, 0, True, 1, 4, mhsa(4, 64, 64, 24)],
            [160, 160, 3, 3, True, 1, 4, mhsa(4, 64, 64, 24)],
            [160, 160, 3, 0, True, 1, 4]
        ]
    },
    "layer4": {
        "block_name": "uib",
        "num_blocks": 12,
        "block_specs": [
            [160, 256, 5, 5, True, 2, 6],
            [256, 256, 5, 5, True, 1, 4],
            [256, 256, 3, 5, True, 1, 4],
            [256, 256, 3, 5, True, 1, 4],
            [256, 256, 0, 0, True, 1, 2],
            [256, 256, 3, 5, True, 1, 2],
            [256, 256, 0, 0, True, 1, 2],
            [256, 256, 0, 0, True, 1, 4, mhsa(4, 64, 64, 12)],
            [256, 256, 3, 0, True, 1, 4, mhsa(4, 64, 64, 12)],
            [256, 256, 5, 5, True, 1, 4, mhsa(4, 64, 64, 12)],
            [256, 256, 5, 0, True, 1, 4, mhsa(4, 64, 64, 12)],
            [256, 256, 5, 0, True, 1, 4]
        ]
    },  
    "layer5": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [256, 960, 1, 1],
            [960, 1280, 1, 1]
        ]
    }
}

MNV4HybridConvLarge_BLOCK_SPECS = {
    "conv0": {
        "block_name": "convbn",
        "num_blocks": 1,
        "block_specs": [
            [3, 24, 3, 2]
        ]
    },
    "layer1": {
        "block_name": "fused_ib",
        "num_blocks": 1,
        "block_specs": [
            [24, 48, 2, 4.0, True]
        ]
    },
    "layer2": {
        "block_name": "uib",
        "num_blocks": 2,
        "block_specs": [
            [48, 96, 3, 5, True, 2, 4],
            [96, 96, 3, 3, True, 1, 4]
        ]
    },
    "layer3": {
        "block_name": "uib",
        "num_blocks": 11,
        "block_specs": [
            [96,  192, 3, 5, True, 2, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 3, True, 1, 4],
            [192, 192, 3, 5, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4],
            [192, 192, 5, 3, True, 1, 4, mhsa(8, 48, 48, 24)],
            [192, 192, 5, 3, True, 1, 4, mhsa(8, 48, 48, 24)],
            [192, 192, 5, 3, True, 1, 4, mhsa(8, 48, 48, 24)],
            [192, 192, 5, 3, True, 1, 4, mhsa(8, 48, 48, 24)],
            [192, 192, 3, 0, True, 1, 4]
        ]
    },
    "layer4": {
        "block_name": "uib",
        "num_blocks": 14,
        "block_specs": [
            [192, 512, 5, 5, True, 2, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 3, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 0, True, 1, 4],
            [512, 512, 5, 3, True, 1, 4],
            [512, 512, 5, 5, True, 1, 4, mhsa(8, 64, 64, 12)],
            [512, 512, 5, 0, True, 1, 4, mhsa(8, 64, 64, 12)],
            [512, 512, 5, 0, True, 1, 4, mhsa(8, 64, 64, 12)],
            [512, 512, 5, 0, True, 1, 4, mhsa(8, 64, 64, 12)],
            [512, 512, 5, 0, True, 1, 4]
        ]
    },  
    "layer5": {
        "block_name": "convbn",
        "num_blocks": 2,
        "block_specs": [
            [512, 960, 1, 1],
            [960, 1280, 1, 1]
        ]
    }
}

MODEL_SPECS = {
    "MobileNetV4ConvSmall": MNV4ConvSmall_BLOCK_SPECS,
    "MobileNetV4ConvMedium": MNV4ConvMedium_BLOCK_SPECS,
    "MobileNetV4ConvLarge": MNV4ConvLarge_BLOCK_SPECS,
    "MobileNetV4HybridMedium": MNV4HybridConvMedium_BLOCK_SPECS,
    "MobileNetV4HybridLarge": MNV4HybridConvLarge_BLOCK_SPECS
}

In [83]:
convnum = 0
convadd = 0
convin = [64]
convd = [9]
convint = []
convoup = []
convkernel = []
convstrid = []
convpadding = []
convgroup = []

# Support ['MobileNetV4ConvSmall', 'MobileNetV4ConvMedium', 'MobileNetV4ConvLarge']
# Also supported ['MobileNetV4HybridMedium', 'MobileNetV4HybridLarge']
model = MobileNetV4("MobileNetV4ConvSmall")

# Check the trainable params
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

# Check the model's output shape
print("Check output shape ...")
x = torch.rand(4, 1, 9, 64, 64)
y = model(x)
for i in y:
    print(i.shape)

Number of parameters: 2792032
Check output shape ...
conv2d torch.Size([4, 32, 5, 112, 112])
torch.Size([4, 32, 3, 56, 56])
torch.Size([4, 64, 2, 28, 28])
torch.Size([4, 96, 1, 14, 14])
torch.Size([4, 128, 1, 7, 7])
torch.Size([4, 1280, 1, 1, 1])


In [34]:
import math
def calculate_output_size(H_in, W_in, kernel_size, stride, padding, dilation=1):
    H_out = math.floor((H_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    W_out = math.floor((W_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    return H_out, W_out

def conv3d_output_shape(D_in, H_in, kernel_size, stride, padding, dilation=1):
    D_out = math.floor((D_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    H_out = math.floor((H_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    return D_out, H_out, H_out  # D_out, H_out, W_out are the same


In [35]:
convin=[224]
for i in range(len(convint)):
    a, b, c= conv3d_output_shape(convd[i], convin[i], convkernel[i], convstrid[i], convpadding[i])
    convd.append(a)
    convin.append(b)
print(len(convin))
print(len(convkernel))

47
46


In [42]:
print(convd)

[9, 5, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [40]:
import math
import torch.nn as nn

def conv2d_output_shape(H_in, W_in, kernel_size, stride, padding):
    H_out = math.floor((H_in + 2 * padding - kernel_size) / stride + 1)
    W_out = math.floor((W_in + 2 * padding - kernel_size) / stride + 1)
    return H_out, W_out
def conv3d_output_shape(D_in, H_in, kernel_size, stride, padding, dilation=1):
    D_out = math.floor((D_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    H_out = math.floor((H_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    return D_out, H_out, H_out  # D_out, H_out, W_out are the same

def conv2d_flops(H_in, W_in, inp, oup, kernel_size, stride, groups):
    padding = (kernel_size - 1) // 2
    H_out, W_out = conv2d_output_shape(H_in, W_in, kernel_size, stride, padding)
    flops = H_out * W_out * oup * kernel_size * kernel_size * (inp // groups)
    print("HOUT:", H_out)
    return flops
def conv3d_flops(D_in, H_in, inp, oup, kernel_size, stride, padding, groups):
    D_out, H_out, W_out = conv3d_output_shape(D_in, H_in, kernel_size, stride, padding)
    print("Dout: {}, Hout: {}".format(D_out, H_out))
    return D_out * H_out * W_out * oup * kernel_size * kernel_size * kernel_size * (inp // groups)

# Example usage
H_in, W_in = 64, 64  # Example input dimensions
inp = 1              # Example input channels
oup = 32             # Example output channels
kernel_size = 3
stride = 2
groups = 1

flops = conv2d_flops(H_in, W_in, inp, oup, kernel_size, stride, groups)
print(f"Conv2D FLOPs: {flops}")


HOUT: 32
Conv2D FLOPs: 294912


In [41]:
convadd = 0
convnum = 0
for i in range(len(convint)):
    print('#',convnum)
    print("input Depth: {0}, input demension: {1}*{1}, input channel: {2}, output channel: {3}, kernel: {4}, stride: {5}, group: {6}".format(convd[i], convin[i], convint[i], convoup[i], convkernel[i], convstrid[i], convpadding[i], convgroup[i]))
    flops = conv3d_flops(convd[i], convin[i], convint[i], convoup[i], convkernel[i], convstrid[i], convpadding[i], convgroup[i])
    print(flops)
    print()
    convadd += flops
    convnum+=1
print(convadd)

# 0
input Depth: 9, input demension: 224*224, input channel: 1, output channel: 32, kernel: 3, stride: 2, group: 1
Dout: 5, Hout: 112
54190080

# 1
input Depth: 5, input demension: 112*112, input channel: 32, output channel: 32, kernel: 3, stride: 2, group: 1
Dout: 3, Hout: 56
260112384

# 2
input Depth: 3, input demension: 56*56, input channel: 32, output channel: 32, kernel: 1, stride: 1, group: 0
Dout: 3, Hout: 56
9633792

# 3
input Depth: 3, input demension: 56*56, input channel: 32, output channel: 96, kernel: 3, stride: 2, group: 1
Dout: 2, Hout: 28
130056192

# 4
input Depth: 2, input demension: 28*28, input channel: 96, output channel: 64, kernel: 1, stride: 1, group: 0
Dout: 2, Hout: 28
9633792

# 5
input Depth: 2, input demension: 28*28, input channel: 64, output channel: 64, kernel: 5, stride: 1, group: 2
Dout: 2, Hout: 28
12544000

# 6
input Depth: 2, input demension: 28*28, input channel: 64, output channel: 192, kernel: 1, stride: 1, group: 0
Dout: 2, Hout: 28
19267584

#

In [168]:
print(len(convint))

46
